# AllegroGraph Security using the Python API

It is possible to install the required packaged with either of the following commands (depending on your package manager)

In [ ]:
!pip install agraph-python

We will only set the environment variables for `AGRAPH_PASSWORD`, `AGRAPH_HOST` and `AGRAPH_PORT`. You will not need to change the port, but please copy the host from your server. We have left a (nonworking) example in place

In [1]:
import os
os.environ['AGRAPH_PASSWORD'] = 'GrMEKDvQFaN2bkrHJeiCbv'
os.environ['AGRAPH_HOST'] = 'https://ag197y8xsj2epl2e.allegrograph.cloud'
os.environ['AGRAPH_PORT'] = '443'

Here we import the required packages and define a function that executed a query and returns a pandas dataframe

In [2]:
from franz.openrdf.sail.allegrographserver import AllegroGraphServer as ag_server
from franz.openrdf.connect import ag_connect
from franz.openrdf.query.query import QueryLanguage
from franz.openrdf.vocabulary import RDF, RDFS
from pprint import pprint

def rsp(conn, query_string):
    with conn.executeTupleQuery(query_string) as result:
        df = result.toPandas()
    df = df.reindex(columns=['s','p','o'])
    return df.head(25)

# Strategy 1: RBAC on the Repository Level
We start with the most basic security method, limiting access per user or role per AllegroGraph repository. Note that as the `admin` user you have **superuser** privileges.

First we create some new repositories and populate with some data. For later examples take note that we add a fourth element to each triple as well

In [3]:
conn = ag_connect('study1', create=True, clear=True, user='admin')

#create a namespace
f = conn.namespace('http://franz.com/')
conn.setNamespace('f', 'http://franz.com/')

triples_study_1 = [
    (f.study1, RDF.TYPE, f.Study, f.study1),
    (f.study1, RDFS.LABEL, "Study 1", f.study1),
    (f.study1, f.hasTopic, "Why the number 1 is the most beautiful number", f.study1)]
conn.addTriples(triples_study_1)
print(conn.size())

3


Now will do the same for a new different repo

In [4]:
conn = ag_connect('study2', create=True, clear=True, user='admin')

#create a namespace
f = conn.namespace('http://franz.com/')
conn.setNamespace('f', 'http://franz.com/')

triples_study_2 = [
    (f.study2, RDF.TYPE, f.Study, f.study2),
    (f.study2, RDFS.LABEL, "Study 2", f.study2),
    (f.study2, f.hasTopic, "Why 2 is better than 1", f.study2)]
conn.addTriples(triples_study_2)
print(conn.size())

3


Create a server connection (different from `ag_connect` and establishing a connection to a specific repo)

In [5]:
server = ag_server(user='admin')

Look at existing users

In [6]:
for user in server.listUsers():
    print(user)

admin
superuser


Now we create two new users

In [7]:
try:
    server.addUser('user1', 'pw1')
    server.addUser('user2', 'pw2')
except:
    pass
    
for user in server.listUsers():
    print(user)

admin
superuser
user1
user2


We will allow *user1* to look at `study1` and allow *user2* to look at both `study1` and `study2`. Remember that we are currently the *superuser*

In [8]:
server.addUserAccess('user1', read=True, write=True, catalog='/', repository='study1')
server.addUserAccess('user2', read=True, write=True, catalog='/', repository='study1')
server.addUserAccess('user2', read=True, write=True, catalog='/', repository='study2')

pprint(f"User1: {server.listUserAccess('user1')}")
pprint(f"User2: {server.listUserAccess('user2')}")

("User1: [{'read': True, 'write': True, 'catalog': '/', 'repository': "
 "'study1', 'query-results-limit': False}]")
("User2: [{'read': True, 'write': True, 'catalog': '/', 'repository': "
 "'study1', 'query-results-limit': False}, {'read': True, 'write': True, "
 "'catalog': '/', 'repository': 'study2', 'query-results-limit': False}]")


Now we will try to have *user1* query both `study1` and `study2`

In [9]:
conn1 = ag_connect('study1', user='user1', password='pw1')

rsp(conn1, "select ?s ?p ?o where { ?s ?p ?o }")

,s,p,o
0,<http://franz.com/study1>,<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>,<http://franz.com/Study>
1,<http://franz.com/study1>,<http://www.w3.org/2000/01/rdf-schema#label>,Study 1
2,<http://franz.com/study1>,<http://franz.com/hasTopic>,Why the number 1 is the most beautiful number


In [10]:
try:
    conn2 = ag_connect('study2', user='user1', password='pw1')

    rsp(conn2, "select ?s ?p ?o where { ?s ?p ?o }")
except:
    pass

401 User 'user1' does not have sufficient permissions to perform this operation.


*User2* is able to query both `study1` and `study2`

In [11]:
conn1 = ag_connect('study1', user='user2', password='pw2')

rsp(conn1, "select ?s ?p ?o where { ?s ?p ?o }")

,s,p,o
0,<http://franz.com/study1>,<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>,<http://franz.com/Study>
1,<http://franz.com/study1>,<http://www.w3.org/2000/01/rdf-schema#label>,Study 1
2,<http://franz.com/study1>,<http://franz.com/hasTopic>,Why the number 1 is the most beautiful number


We revoke access for all users (For the `server` connection we are still connected as *superuser*)

In [12]:
server.deleteUserAccess('user1', read=True, write=True, catalog='/', repository='study1') 
server.deleteUserAccess('user2', read=True, write=True, catalog='/', repository='study1')
server.deleteUserAccess('user2', read=True, write=True, catalog='/', repository='study2')

pprint(f"User1: {server.listUserAccess('user1')}")
pprint(f"User2: {server.listUserAccess('user2')}")

'User1: []'
'User2: []'


Now we will create some roles and list them

In [13]:
try:
    role1 = server.addRole('role1')
    role2 = server.addRole('role2')
except:
    print('User already exists')

for role in server.listRoles():
    print(role)

role1
role2


Connect the roles to different repositories

In [14]:
server.addRoleAccess('role1', read=True, write=True, catalog='/', repository='study1')
server.addRoleAccess('role2', read=True, write=True, catalog='/', repository='study2')

print(f"Role1: {server.listRoleAccess('role1')}")
print(f"Role2: {server.listRoleAccess('role2')}")

Role1: [{'read': True, 'write': True, 'catalog': '/', 'repository': 'study1', 'query-results-limit': False}]
Role2: [{'read': True, 'write': True, 'catalog': '/', 'repository': 'study2', 'query-results-limit': False}]


Now we try to query `study1` as *user1* and show it fails

In [15]:
try:
    conn2 = ag_connect('study1', user='user1', password='pw1')

    rsp(conn2, "select ?s ?p ?o where { ?s ?p ?o }")
except:
    pass

Let's add *user1* to role1 and show that the query now works

In [16]:
server.addUserRole('user1', 'role1')

conn2 = ag_connect('study1', user='user1', password='pw1')
rsp(conn2, "select ?s ?p ?o where { ?s ?p ?o }")

,s,p,o
0,<http://franz.com/study1>,<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>,<http://franz.com/Study>
1,<http://franz.com/study1>,<http://www.w3.org/2000/01/rdf-schema#label>,Study 1
2,<http://franz.com/study1>,<http://franz.com/hasTopic>,Why the number 1 is the most beautiful number


Finish off the first example by first revoking access for the roles, and then deleting them

In [17]:
try:
    server.deleteRoleAccess('role1', read=True, write=True, catalog='/', repository='study1')
    server.deleteRoleAccess('role2', read=True, write=True, catalog='/', repository='study2')
except:
    pass

print(f"Role1: {server.listRoleAccess('role1')}")
print(f"Role2: {server.listRoleAccess('role2')}")

try:
    server.deleteRole('role1')
    server.deleteRole('role2')
except:
    pass

for role in server.listRoles():
    print(role)

Role1: []
Role2: []


# Strategy 2: RBAC on the Level of Graphs (or any Part of the Triple)

First we create a single repository that combines both sets of triples used in the previous examples. Notice that triples about `study1` has graph `<http://franz.com/Study1>` and triples about `study2` has graph `<http://franz.com/Study2>`

In [18]:
conn = ag_connect('studies', create=True, clear=True, user='admin')

#create a namespace
f = conn.namespace('http://franz.com/')

conn.addTriples(triples_study_1)
conn.addTriples(triples_study_2)
print(conn.size())

6


Create a new user, role, with permissions on reading and writing on new `studies` repo

In [19]:
try: server.addUser('user3', 'pw3')
except: pass
server.addRole('role3')
server.addUserRole('user3', 'role3')
server.addRoleAccess('role3', read=True, write=True, catalog='/', repository='studies')

conn = ag_connect('studies', user='user3', password='pw3')
rsp(conn, "select ?s ?p ?o where { ?s ?p ?o }")

,s,p,o
0,<http://franz.com/study1>,<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>,<http://franz.com/Study>
1,<http://franz.com/study1>,<http://www.w3.org/2000/01/rdf-schema#label>,Study 1
2,<http://franz.com/study1>,<http://franz.com/hasTopic>,Why the number 1 is the most beautiful number
3,<http://franz.com/study2>,<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>,<http://franz.com/Study>
4,<http://franz.com/study2>,<http://www.w3.org/2000/01/rdf-schema#label>,Study 2
5,<http://franz.com/study2>,<http://franz.com/hasTopic>,Why 2 is better than 1


Add a Role Security Filter so that *role3* can only see triples with graph `<http://franz.com/Study1>`

In [20]:
server.addRoleSecurityFilter('role3', 'allow', s=None, p=None, o=None, g=f.study1)

In [21]:
rsp(conn, "select ?s ?p ?o where { ?s ?p ?o }")

,s,p,o
0,<http://franz.com/study1>,<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>,<http://franz.com/Study>
1,<http://franz.com/study1>,<http://www.w3.org/2000/01/rdf-schema#label>,Study 1
2,<http://franz.com/study1>,<http://franz.com/hasTopic>,Why the number 1 is the most beautiful number
